In [2]:
import numpy as np

In [3]:
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None, value=None):
        
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [12]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    
    def fit(self, X, y):
        self.root = self.grow_tree(X, y)
    
    def gini_impurity(self, y):
        _, counts = np.unique(y, return_counts = True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)
    
    def information_gain(self, parent, left_child, right_child):
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        
        return (self.gini_impurity(parent) - (weight_left * self.gini_impurity(left_child) +
                                             weight_right * self.gini_impurity(right_child)))
    def best_split(self, X, y):
        best_gain = -1
        best_feature, best_threshold = None, None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <=threshold
                right_mask = ~left_mask
                gain = self.information_gain(y, y[left_mask], y[right_mask])
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold
    
    def grow_tree(self, X, y, depth = 0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        if(depth == self.max_depth or n_samples  < 2 or n_classes == 1):
            return Node(value = np.argmax(np.bincount(y)))
        
        
        feature, threshold = self.best_split(X, y)
        left_mask = X[:, feature] <=threshold
        right_mask = ~left_mask
        left = self.grow_tree(X[left_mask], y[left_mask], depth+1)
        right = self.grow_tree(X[right_mask], y[right_mask], depth+1)
        
        return Node(feature = feature, threshold=threshold, left = left, right = right)
    
    def predict(self, X):
        return np.array([self.traverse_tree(x, self.root) for x in X])
    
    def traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        else:
            return self.traverse_tree(x, node.right)

In [13]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([0,0,1,1])
tree = DecisionTree(max_depth = 3)
tree.fit(X, y)
predictions = tree.predict(X)

In [14]:
predictions

array([0, 0, 1, 1], dtype=int64)

In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [16]:
iris = load_iris()
X, y = iris.data, iris.target

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
tree = DecisionTree(max_depth = 3)
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

In [19]:
predictions

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0], dtype=int64)

In [20]:
accuracy = np.mean(predictions == y_test)

In [21]:
accuracy

1.0